In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import fastf1
from src.plotset import setup_plot
from fastf1 import plotting
import statsmodels.api as sm
from collections import defaultdict

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
# -------------------------
# Data containers
# -------------------------

# Per driver: all stints (compound-level info)
driver_consistency = defaultdict(list)  
# Example: { 'NOR': [('SOFT', 18, 0.09), ('MEDIUM', 22, 0.05), ...] }

# Per driver per race: all stints (to compute race-wise overall consistency)
race_consistency = defaultdict(lambda: defaultdict(list))
# Example: { 'NOR': {1: [(compound, n, stddev), (compound, n, stddev)], 2: [...]} }

# -------------------------
# Data collection
# -------------------------

for rnd in range(1, 15):  # Races 1 to 14
    if rnd in [1,12,13]:
        continue
    print(f"Processing Race {rnd}...")
    session = fastf1.get_session(2025, rnd, 'R')
    session.load(laps=True, telemetry=False, weather=False, messages=False)
    
    for drv in session.drivers:
        drv_abbr = session.get_driver(drv)['Abbreviation']
        laps = session.laps.pick_drivers(drv)[[
            'LapNumber', 'LapTime', 'Stint', 'Compound', 'TyreLife',
            'TrackStatus', 'PitInTime', 'PitOutTime'
        ]].pick_quicklaps()
        
        # Remove laps with yellow/red flags (track status 4)
        laps = laps[~(laps.TrackStatus.str.contains('4'))]
        
        for stint_num, stint_df in laps.groupby('Stint'):
            if len(stint_df) < 10:  # Skip very short stints
                continue
            
            stint_df = stint_df[(stint_df.PitInTime.isnull()) & (stint_df.PitOutTime.isnull())].copy()
            if stint_df.empty:
                continue
            
            stint_df['LapTime'] = stint_df.LapTime.dt.total_seconds()
            
            x = sm.add_constant(stint_df['TyreLife'])
            y = stint_df['LapTime']
            
            try:
                model = sm.OLS(y, x).fit()
                stddev = model.resid.std()
                
                compound = stint_df['Compound'].iloc[0]
                n_laps = len(stint_df)
                
                # Store globally
                driver_consistency[drv_abbr].append((compound, n_laps, stddev))
                
                # Store per race
                race_consistency[drv_abbr][rnd].append((compound, n_laps, stddev))
                
            except Exception as e:
                print(f"Error for {drv_abbr} Race {rnd} stint {stint_num}: {e}")

In [ ]:
# -------------------------
# Post-processing functions
# -------------------------

def pooled_stddev(values):
    """
    values: list of (n_laps, stddev)
    Returns pooled stddev
    """
    if not values:
        return np.nan
    num = sum((n-1)*(s**2) for n, s in values)
    den = sum((n-1) for n, s in values)
    return np.sqrt(num/den) if den > 0 else np.nan

def overall_driver_consistency(driver_consistency):
    """
    Returns dict {driver: overall stddev}
    """
    results = {}
    for drv, stints in driver_consistency.items():
        vals = [(n, s) for _, n, s in stints]
        results[drv] = pooled_stddev(vals)
    return results

def compound_wise_consistency(driver_consistency):
    """
    Returns dict {driver: {compound: stddev}}
    """
    results = {}
    for drv, stints in driver_consistency.items():
        comp_dict = defaultdict(list)
        for comp, n, s in stints:
            comp_dict[comp].append((n, s))
        results[drv] = {comp: pooled_stddev(vals) for comp, vals in comp_dict.items()}
    return results

def race_wise_consistency(race_consistency):
    """
    Returns dict {driver: {race: stddev}}
    """
    results = {}
    for drv, races in race_consistency.items():
        race_dict = {}
        for rnd, stints in races.items():
            vals = [(n, s) for _, n, s in stints]
            race_dict[rnd] = pooled_stddev(vals)
        results[drv] = race_dict
    return results

# -------------------------
# Example usage
# -------------------------

overall = overall_driver_consistency(driver_consistency)
compoundwise = compound_wise_consistency(driver_consistency)
racewise = race_wise_consistency(race_consistency)

In [ ]:
overall_df = pd.Series(overall).sort_values()

In [ ]:
compoundwise_df = pd.DataFrame(compoundwise).T

In [ ]:
racewise_df = pd.DataFrame(racewise)